## Conisidering how to best provide general landscape of methylation.

Here is what was used for methylkit
```
file.list_18=list('../data/zr1394_1_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_2_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_3_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',   '../data/zr1394_4_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_5_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',                              '../data/zr1394_6_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_7_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_8_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',               '../data/zr1394_9_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_10_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',                              '../data/zr1394_11_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_12_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_13_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',                         '../data/zr1394_14_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_15_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',                              '../data/zr1394_16_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_17_s456_trimmed_bismark_bt2.deduplicated.sorted.bam',
'../data/zr1394_18_s456_trimmed_bismark_bt2.deduplicated.sorted.bam'
)
```



Lets see if files for bismark are still on mox

`reads_dir="/gscratch/srlab/sr320/data/olurida-bs/"`

Made single concat file and bismarked.

```
#!/bin/bash
## Job Name
#SBATCH --job-name=oly-gen
## Allocation Definition
#SBATCH --account=srlab
#SBATCH --partition=srlab
## Resources
## Nodes (We only get 1, so this is fixed)
#SBATCH --nodes=1
## Walltime (days-hours:minutes:seconds format)
#SBATCH --time=00-100:00:00
## Memory per node
#SBATCH --mem=100G
#SBATCH --mail-type=ALL
#SBATCH --mail-user=sr320@uw.edu
## Specify the working directory for this job
#SBATCH --workdir=/gscratch/srlab/sr320/analyses/2019/0620
 
 
 
# Directories and programs
wd=$(pwd)
bismark_dir="/gscratch/srlab/programs/Bismark-0.21.0"
bowtie2_dir="/gscratch/srlab/programs/bowtie2-2.3.4.1-linux-x86_64/"
samtools="/gscratch/srlab/programs/samtools-1.9/samtools"
#reads_dir="/gscratch/srlab/sr320/data/olurida-bs/decomp/"
 
 
 
source /gscratch/srlab/programs/scripts/paths.sh
 
 
${bismark_dir}/bismark \
--path_to_bowtie ${bowtie2_dir} \
-genome /gscratch/srlab/sr320/data/olurida-genomes/v081 \
-p 14 \
--non_directional \
/gscratch/scrubbed/sr320/0620/zr1394_all_s456_trimmed.fq
 
 

find *.bam | \
xargs basename -s .bam | \
xargs -I{} ${bismark_dir}/deduplicate_bismark \
--bam \
--single \
{}.bam



${bismark_dir}/bismark_methylation_extractor \
--bedGraph --counts --scaffolds \
--multicore 14 \
--buffer_size 75% \
*deduplicated.bam



# Bismark processing report

${bismark_dir}/bismark2report

#Bismark summary report

${bismark_dir}/bismark2summary



# Sort files for methylkit and IGV

find *deduplicated.bam | \
xargs basename -s .bam | \
xargs -I{} ${samtools} \
sort --threads 28 {}.bam \
-o {}.sorted.bam

# Index sorted files for IGV
# The "-@ 28" below specifies number of CPU threads to use.

find *.sorted.bam | \
xargs basename -s .sorted.bam | \
xargs -I{} ${samtools} \
index -@ 28 {}.sorted.bam
```